<a href="https://colab.research.google.com/github/RP335/microtonal_experiments/blob/main/microtonal_parrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generator for 180 midi files for the dataset. different kinds of edo classes and only triads.

In [1]:
!pip install mido

import mido
import os


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


Imported all the required stuff for processing. Now we'd be moving over to generating midi triads

In [2]:
# Function to create MIDI files
def create_microtonal_midi(root_pitch, edo_steps, filename):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    # Convert EDO steps to MIDI pitch
    microtonal_pitches = [root_pitch + step for step in edo_steps]

    for pitch in microtonal_pitches:
        track.append(mido.Message('note_on', note=pitch, velocity=64, time=0))
        track.append(mido.Message('note_off', note=pitch, velocity=64, time=480))

    mid.save(filename)

# I am going to be generating triads
# Define EDOs and number of files to generate per EDO (20 times 9 is 180 that's our dataset size)
edos = [5, 9, 31, 21, 19, 24, 26, 7, 13]
num_files_per_edo = 20
root_pitch = 60  # Middle C

output_dir_midi = 'microtonal_midi_files'
os.makedirs(output_dir_midi, exist_ok=True)

# Generate MIDI files
for edo in edos:
    for i in range(num_files_per_edo):
        filename = os.path.join(output_dir_midi, f'{edo}edo_chord_{i+1}.mid')
        edo_steps = [int(edo * step / 12) for step in range(3)]
        create_microtonal_midi(root_pitch, edo_steps, filename)

print(f'Generated {num_files_per_edo * len(edos)} MIDI files in {output_dir_midi} directory.')

Generated 180 MIDI files in microtonal_midi_files directory.


In [3]:
!apt-get install fluidsynth
!pip install midi2audio

from midi2audio import FluidSynth

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icccm4 libxcb-image0
  libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icc

Imported our things to convert these midi into audio

In [4]:
soundfont_path = '/usr/share/sounds/sf2/FluidR3_GM.sf2'
output_dir_audio = 'microtonal_audio_files'
fs = FluidSynth(soundfont_path)

# Convert MIDI files to audio
for edo in edos:
    for i in range(num_files_per_edo):
        midi_file = os.path.join(output_dir_audio, f'{edo}edo_chord_{i+1}.mid')
        audio_file = os.path.join(output_dir_audio, f'{edo}edo_chord_{i+1}.wav')
        fs.midi_to_audio(midi_file, audio_file)

print(f'Converted all MIDI files to audio in {output_dir_audio} directory.')

Converted all MIDI files to audio in microtonal_midi_files directory.


In [5]:
!rm -rf